# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset represents 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LinearRegression

import seaborn as sns

from sklearn.preprocessing import StandardScaler,OneHotEncoder, OrdinalEncoder, LabelEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from sklearn.tree import export_text, plot_tree,export_graphviz

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel

from sklearn import set_config
set_config(display='diagram')
     
from sklearn.dummy import DummyClassifier


In [5]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [6]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [25]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [12]:
df['y'] = df['y'].map({"yes":1,"no":0})

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Objectives: to classify if a customer will subscribe a term deposit based on past campaign information like customer age, job, marital status etc.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [17]:
df1 = df[['age','job','marital','education','default','housing','loan','y']]

In [18]:
dummies = pd.get_dummies(df1.select_dtypes(include = 'object'))
data_w_dummies = pd.concat([df1, dummies], axis=1)
df_w_d = data_w_dummies.drop(columns = df1.select_dtypes(include = 'object').columns)

In [21]:
df_w_d['y'] = df_w_d['y'].map({1:'yes',0:'no'})

In [22]:
df_w_d

,age,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
0,56,no,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,57,no,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
2,37,no,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,40,no,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,56,no,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,yes,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41184,46,no,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
41185,56,no,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41186,44,yes,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [23]:
df_w_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   age                            41188 non-null  int64 
 1   y                              41188 non-null  object
 2   job_admin.                     41188 non-null  uint8 
 3   job_blue-collar                41188 non-null  uint8 
 4   job_entrepreneur               41188 non-null  uint8 
 5   job_housemaid                  41188 non-null  uint8 
 6   job_management                 41188 non-null  uint8 
 7   job_retired                    41188 non-null  uint8 
 8   job_self-employed              41188 non-null  uint8 
 9   job_services                   41188 non-null  uint8 
 10  job_student                    41188 non-null  uint8 
 11  job_technician                 41188 non-null  uint8 
 12  job_unemployed                 41188 non-null  uint8 
 13  j

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [27]:
X = df_w_d.drop(['y'], axis=1)
y=df_w_d['y']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [32]:
dummy_clf = DummyClassifier(strategy='stratified')
dummy_clf.fit(X,y)

DummyClassifier(strategy='stratified')

In [34]:
## Model Accuracy
dummy_clf.score(X,y)

0.8007672137515781

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [42]:
start_time = time.time()

lgr_pipe = Pipeline([('lgr', LogisticRegression(random_state=42, max_iter = 1000))])

lgr_pipe.fit(X_train, y_train)

end_time = time.time()

lgr_time = end_time - start_time

print(lgr_time)

1.1263701915740967


### Problem 9: Score the Model

What is the accuracy of your model?

In [38]:
pipe_1_acc = lgr_pipe.score(X_test, y_test)
pipe_2_acc = lgr_pipe.score(X_train, y_train)
print(pipe_1_acc)
print(pipe_2_acc)

0.8875940762320952
0.8872394297804447


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [43]:
start_time = time.time()

knn_pipe=Pipeline([('knn', KNeighborsClassifier())])

knn_pipe.fit(X_train, y_train)

end_time = time.time()

knn_time = end_time - start_time

print(knn_time)

0.0660405158996582


In [44]:
knn_1_acc = knn_pipe.score(X_test, y_test)
knn_2_acc = knn_pipe.score(X_train, y_train)
print(knn_1_acc)
print(knn_2_acc)

0.8805535324107793
0.8908813429988554


In [45]:
start_time = time.time()

dtree_pipe=Pipeline([('dtree',DecisionTreeClassifier())])

dtree_pipe.fit(X_train, y_train)

end_time = time.time()

dtree_time = end_time - start_time

print(dtree_time)

0.4186272621154785


In [46]:
dtree_1_acc = dtree_pipe.score(X_test, y_test)
dtree_2_acc = dtree_pipe.score(X_train, y_train)
print(dtree_1_acc)
print(dtree_2_acc)

0.8617787488872704
0.9179702403662724


In [47]:
start_time = time.time()

svc_pipe=Pipeline([('svc',SVC())])

svc_pipe.fit(X_train, y_train)

end_time = time.time()

svc_time = end_time - start_time

print(svc_time)

30.51779079437256


In [48]:
svc_1_acc = svc_pipe.score(X_test, y_test)
svc_2_acc = svc_pipe.score(X_train, y_train)
print(svc_1_acc)
print(svc_2_acc)

0.8875940762320952
0.8872394297804447


In [50]:
final = pd.DataFrame(columns=["Model", "Train Time","Train Accuracy",'Test Accuracy'])

entry = pd.DataFrame.from_dict({
     "Model": ["Logistic",'KNN','Decision Tree','SVM'],
     "Train Time":  [lgr_time, knn_time, dtree_time,svc_time],
     "Train Accuracy": [pipe_2_acc,knn_2_acc,dtree_2_acc,svc_2_acc],
     'Test Accuracy':[pipe_1_acc,knn_1_acc,dtree_1_acc,svc_1_acc]
})

df_f = pd.concat([final, entry], ignore_index=True)

df_f

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic,1.126370,0.887239,0.887594
1,KNN,0.066041,0.890881,0.880554
2,Decision Tree,0.418627,0.917970,0.861779
3,SVM,30.517791,0.887239,0.887594


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [65]:
#Knn
knn_params={'knn__n_neighbors':range(1,5),
            'knn__weights':['uniform','distance']}

#Decision tree
dtree_params = {'dtree__max_depth':[2,4,6],
                'dtree__min_samples_split':[0.1, 0.2, 0.05],
                'dtree__criterion': ['gini','entropy'],
                'dtree__min_samples_leaf':[1,2,3]}


#SVM
svc_params= {'svc__kernel': ['rbf'],
              'svc__gamma': [0.1, 1.0]}

In [56]:
knn_grid=GridSearchCV(estimator=knn_pipe,
                      param_grid=knn_params,
                     cv=5)
knn=knn_grid.fit(X_train,y_train)

In [58]:
knn.best_params_

{'knn__n_neighbors': 4, 'knn__weights': 'uniform'}

In [57]:
knn.best_score_

0.882522300113056

In [59]:
dtree_grid=GridSearchCV(estimator=dtree_pipe,
                      param_grid=dtree_params,
                      cv=5)
dtree=dtree_grid.fit(X_train,y_train)

In [60]:
dtree.best_params_

{'dtree__criterion': 'gini',
 'dtree__max_depth': 2,
 'dtree__min_samples_leaf': 1,
 'dtree__min_samples_split': 0.1}

In [61]:
dtree.best_score_

0.8872394340495436

In [66]:
svc_grid=GridSearchCV(estimator=svc_pipe,
                      param_grid=svc_params,
                      cv=5)
svm=svc_grid.fit(X_train,y_train)

In [67]:
svm.best_params_

{'svc__gamma': 0.1, 'svc__kernel': 'rbf'}

In [68]:
svm.best_score_

0.8872047419440847

Conclusion

Taking the seven features, models gives a classification estimation around 90%. Of the four methods, KNN takes the less time and gives good result. SVM takes the longest time and it's accuracy is not significantly higher comparing to other models. Hyperpamater tuning doesn't provide much improvement for the models other than decision tree. Given the increased in tuning and end results, it is recommended to use knn for the classifaction for future customers. 